In [2]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import pandas as pd

In [16]:
# Đọc dữ liệu từ file CSV (giả sử bạn đã có dữ liệu từ tập bạn cung cấp)
df = pd.read_csv('creditcard.csv')
# Tạo một hàm để chia dữ liệu thành các batches (lô dữ liệu)
def create_datastream(data, batch_size):
    datastream = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size]
        datastream.append(batch)
    return datastream

# Gọi hàm để chia dữ liệu thành các batch 100 hàng
batch_size = 1000
S = create_datastream(df, batch_size)

In [15]:
# Function to train Random Forest classifier
def train_random_forest_classifier(X, y, n_estimators=10):
    """
    Trains a Random Forest classifier on X, y data.
    n_estimators: Number of trees in the random forest.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf

TypeError: object of type 'numpy.int32' has no len()

In [ ]:
def adaptive_ensemble_size(C, sample, unique_label, anpha, min_num = 3):
    probability_list = []
    for i in range(len(unique_label)):
        li = [] 
        for j in range(len(C)):
            probabilities = C[0].predict_proba(sample)[0]
            probability_dict = {label: prob for label, prob in zip(C[0].classes_, probabilities)}
            latest_proba = probability_dict.get(unique_label[i], 0)
            probabilities1 = C[j].predict_proba(sample)[0]
            probability_dict1 = {label: prob for label, prob in zip(C[j].classes_, probabilities1)}
            current_proba = probability_dict1.get(unique_label[i], 0)
            if len(li) < min_num:
                li.append(current_proba)
            else:
                if abs(current_proba - latest_proba) < anpha :
                    li.append(current_proba)
                else: 
                    break   
        probability_list.append(li)     
    return probability_list

In [ ]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    for i in range(len(Y)):
        li = probability_list[i]
        
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        
        # Tính hồi quy tuyến tính f(li, g(z))
        slope, intercept = linear_regression(x, y)
        
        # Dự đoán giá trị tiếp theo g(|li| + 1)
        next_value = slope * (len(li) + 1) + intercept
        li.append(next_value)
        
        # Tính xác suất có trọng số bằng phương pháp trung bình có trọng số
        weighted_prob = sum([li[x] * (1 + x * epsilon) for x in range(len(li))]) / len(li)
        predicted_probabilities.append(weighted_prob)
    
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps


In [5]:

def process_data_stream(S, m, k, unique_labels):
    """
    Processes data stream S using a weak Random Forest classifier.
    - S: Data stream (list of data blocks Bi)
    - m: Max size of classifier set C
    - k: Min size of C to make predictions
    """
    C = deque(maxlen=m)  # Pool of classifiers
    pre = []
    for Bi in S:
        block_predictions = []
        X = Bi.iloc[:, :-1]  # Lấy tất cả các cột trừ cột cuối
        y = Bi.iloc[:, -1]   # Lấy cột cuối cùng
        # Train a new classifier for each block
        Ci = train_random_forest_classifier(X, y)
        C.append(Ci)
        while len(C) < k :
            C.append(Ci)
        # For each sample in the block, make predictions
        for index, row in Bi.iterrows():
            # Chuyển sample thành DataFrame với cùng tên cột
            sample = pd.DataFrame([row[:-1]], columns=Bi.columns[:-1])
            anpha = (1500 / len(Bi)) * 0.2
            selected_classifiers = adaptive_ensemble_size(C, sample, unique_labels, anpha)
            pre_sample = tendency_prediction(selected_classifiers, unique_labels)
            block_predictions.append(pre_sample)
        pre.append(block_predictions)
    return C, pre
m = 15
k = 3
unique_labels = set(df.iloc[:, -1])
unique_labels = list(unique_labels)
C, pre = process_data_stream(S, m, k, unique_labels)


Dự đoán lớp: Class 1
